In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# Now you can import from src
from src.data_utils import load_and_process_bike_data

In [19]:
rides = load_and_process_bike_data(years=[2023,2024,2025])

File already exists for 2023-01.
Loading data for 2023-01...
Total records: 1,795,412
Valid records: 22,333
Records dropped: 1,773,079 (98.76%)
Successfully processed data for 2023-01.
File already exists for 2023-02.
Loading data for 2023-02...
Total records: 1,696,171
Valid records: 21,522
Records dropped: 1,674,649 (98.73%)
Successfully processed data for 2023-02.
File already exists for 2023-03.
Loading data for 2023-03...
Total records: 2,118,932
Valid records: 25,938
Records dropped: 2,092,994 (98.78%)
Successfully processed data for 2023-03.
File already exists for 2023-04.
Loading data for 2023-04...
Total records: 2,749,716
Valid records: 27,049
Records dropped: 2,722,667 (99.02%)
Successfully processed data for 2023-04.
File already exists for 2023-05.
Loading data for 2023-05...
Total records: 3,453,152
Valid records: 32,272
Records dropped: 3,420,880 (99.07%)
Successfully processed data for 2023-05.
File already exists for 2023-06.
Loading data for 2023-06...
Total records:

In [20]:
rides

,started_at,start_station_id
18808,2023-01-01 00:10:42.722,6140.05
9820,2023-01-01 00:27:40.052,6822.09
1966,2023-01-01 00:39:13.016,6822.09
8621,2023-01-01 00:51:27.940,6822.09
12356,2023-01-01 01:01:10.818,6140.05
...,...,...
790657,2025-03-31 22:33:09.378,6822.09
797083,2025-03-31 22:38:11.350,6140.05
791329,2025-03-31 23:17:27.478,5905.14
805448,2025-03-31 23:23:22.816,6822.09


In [21]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)
ts_data.head()

,start_hour,start_station_id,rides
0,2023-01-01 00:00:00,5905.140137,0
1,2023-01-01 01:00:00,5905.140137,5
2,2023-01-01 02:00:00,5905.140137,7
3,2023-01-01 03:00:00,5905.140137,3
4,2023-01-01 04:00:00,5905.140137,2


In [26]:
from src.config import PROCESSED_DATA_DIR
ts_data.to_parquet(PROCESSED_DATA_DIR / "ts_data.parquet", engine="pyarrow")

In [22]:
ts_data.shape

(59112, 3)

In [23]:
from src.data_utils import transform_ts_data_info_features_and_target_loop

features, targets = transform_ts_data_info_features_and_target_loop(ts_data, window_size=24*28, step_size=24)

In [24]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,start_hour,start_station_id
0,0,5,7,3,2,1,2,0,2,2,...,18,30,25,13,12,9,9,11,2023-01-29,5905.140137
1,2,0,0,0,1,1,0,0,6,6,...,19,24,17,7,11,14,7,4,2023-01-30,5905.140137
2,1,2,1,0,0,0,1,3,12,12,...,30,40,27,28,23,10,7,7,2023-01-31,5905.140137
3,2,5,0,0,1,0,4,7,34,22,...,28,34,34,22,22,12,17,4,2023-02-01,5905.140137
4,0,2,0,1,1,2,3,5,18,14,...,15,16,23,25,11,10,6,2,2023-02-02,5905.140137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374,4,1,0,1,0,2,8,21,16,11,...,51,70,19,13,8,9,1,5,2025-03-27,6822.089844
2375,0,0,1,0,1,1,3,12,23,6,...,38,61,43,19,11,9,4,6,2025-03-28,6822.089844
2376,3,4,1,1,1,1,2,4,16,9,...,41,49,33,33,8,7,2,4,2025-03-29,6822.089844
2377,0,3,0,0,0,0,1,2,7,10,...,12,12,7,16,6,3,1,7,2025-03-30,6822.089844


In [25]:
tabular_data = features
tabular_data["target"] = targets

from src.config import TRANSFORMED_DATA_DIR
tabular_data.to_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet", engine="pyarrow")